Dou et al. (2020) model solved in parallel

Estimation using Coupled Simulated Annealing, from Xavier et al. (2010).

Agora em Julia versão 1.5.3

This script only solves the model. For a version with unit testing and graphs, refer to "BargainingJulia"



Version to use at HPC - aguia4

Número de simulações = length(pids) * 5

Código para fazer cálculos em duplas

In [1]:
using Distributed

In [2]:
nprocs = 4;

In [3]:
addprocs(nprocs);

In [4]:
@everywhere using DataFrames, LinearAlgebra, Statistics

In [5]:
@everywhere function solve_tree(data, game_parameters, grid=100, hlow=0.01, hhigh=1.0, grid_payment=1000)
    
#     #data ####
#     #all the data is escalated so as to D = 1.0
    Vmax = data[1]; #firm's maximum reorganization value
    L = data[2]; #firm's maximum liquidation value
    Dj = data[3]; #junior's debt face value
    Ds = 1.0 - Dj; #senior's debt face value

    D = Ds + Dj; #total debt, used to escale costs

#     #parameters####
#     #parameters include only the parameters necessary to calculate the model, so it does not include θs0, θj0, μ
    ρ = game_parameters[1]; # (1 - ρ) is the firm's depreciation rate each period
    β = game_parameters[2]; #inverse of the speed of learning
    c0 = game_parameters[3]; #fixed cost of going to court
    λj = game_parameters[4]; #chance of j being called to propose at each period

    #c1 is calibrated. We will start with the same c1 as Dou et al.(2019)
    c1 = c0/30; #variable cost for each period at court
    #Initial definitions: costs, maximum value of the firm, maximum number of periods####


    #number of periods
    function max_turns(Vmax, L, ρ, c0, c1)
        t = 0

        #using the specification of the theoretical model
        #we are not considering the change in indexation from t to t+1 yet

        #the condition L - c0 - c1*t >= 0 assures that the liquidation payoffs won't be negative

        while( (ρ^(t-1) * Vmax > L) && (L - c0 - c1*t >= 0) )
            t = t+1
        end
        return t

    end


    T = max_turns(Vmax, L, ρ, c0, c1)


    #cost function
    function Ct(t, c0=c0, c1=c1)
        #cost at period t=0(index1) is 0
        if(t <= 1)
            return 0
        else
            return c0 * D + c1 * (t-1) * D #test to make index==1 be t==0
        end
    end

    #maximum value of reorganization each period
    function Vt(Vmax, ρ, t)

        if(t <=1)
            return Vmax
        else
            #(t-2) instead of (t-1) because we shifted the indexes in the game so as to include t==0 at index==1
            return ρ^(t-2) * Vmax
        end


    end
    #index 1 corresponds to t=0 and 2 to t=1...

    #liquidation payoffs

    function s_L(t)
        return min(L - Ct(t), Ds)
    end

    function j_L(t)
        return min(L - Ct(t) - s_L(t), Dj)
    end



    #grid size
    grid = grid

    hlow = hlow
    hhigh = hhigh


    hvals = LinRange(hlow, hhigh, grid)



    #array to tell the size of the pie at period t, according to reorganization skill levels
    #arguments: (period, θkt)
    U = Array{Float64}(undef, T+1, grid)


    for t in 1:T+1
        for h in 1:grid
            U[t,h] = hvals[h] * Vt(Vmax, ρ, t) - Ct(t)
        end
    end

    #cumulative distribution function
    function cdf(x, lt, β=β)

        #return the cdf of x given the lower bound lt
        #geq than 1 because of our discretization method 
        if(x >= 1.0)

            return 1.0

        else

            if(x >= lt)
                return 1.0 - ((1.0-x)^β)/((1-lt)^β)
            else
                return 0.0
            end
        end
    end


    #pmf: each point of the grid is at the center of the cdf
    function pmf_cdf(lt, δ = hlow/2, β=β)
        pmf = zeros(grid)

        for (i, h) in enumerate(hvals)
            pmf[i] = cdf(h+δ, lt) - cdf(h-δ, lt)
        end

        return pmf
    end



    #generating the pmfs

    #probability mass function
    pmf = Array{Float64}(undef,grid,grid)

    δ = hlow/2
    for (i, htoday) in enumerate(hvals)
        for (j, htomorrow) in enumerate(hvals)
            pmf[i,j] = cdf(htomorrow+δ, htoday) - cdf(htomorrow-δ, htoday)
        end
    end
    


    #we need a pmf for when we have lkt and want to know θk,t+1
    #we will call it 'pmf2'
    pmf2 = similar(pmf)

    for i in eachindex(hvals)
        for j in eachindex(hvals)
            pmf2[i, j] = sum(pmf[i, :] .* pmf[:, j])
        end
    end



    #continuation values
    #period t (not t+1), θkt, ℓkt, ℓmt
    s_W = zeros(T+1, grid, grid, grid);
    j_W = similar(s_W);


    #optimal payments
    #period, θkt, ℓmt, outputs
    Pst_array = zeros(T, grid, grid, 6); #we won't need payment proposal for period T, a.k.a. "T+1"
    Pjt_array = similar(Pst_array);

    #populating the last period with the liquidation values
    s_W[T+1, :, :, :] .= s_L(T+1); #we need to use .= because it is a broadcast
    j_W[T+1, :, :, :] .= j_L(T+1);

    #function to select the pmf function as it is writen in the paper

    function populate_probm!(t, lmt, probm, pmf, pmf2, grid)
        #takes the indexes of cutoff and lmt

        #at t=0(index 1), all information is symmetric, so we use pmf instead of pmf2
        if(t <= 1)

            for i in 1:grid
                probm[i] = pmf[lmt, i]
            end

        else

            for i in 1:grid
                probm[i] = pmf2[lmt, i]
            end

        end
    end



    #cutoff function: upgrade

    function getcutoff_manual(t, pkt, hkt, m_W, grid)


        cmt=Int64(1)

        while(m_W[t+1, cmt, cmt, hkt] < pkt)
            cmt+=1

            #break if cmt==grid+1
            if(cmt==grid+1)
                return cmt
            end

        end

        return cmt
    end



    #function to "filter" the cutoff

    function cutoff_manual(t, pkt, hkt, lmt, m_W, grid)
        #for now, it will just tell us if the payment offer is negative

        #if payment is negative (waiting offer), the cutoff will be 102 (grid+2), a code for the adversary to always reject it
        if(pkt<0)
             return grid+2
        end

        cmt = getcutoff_manual(t, pkt, hkt, m_W, grid)

        return cmt


    end


    function dot_W(t, hkt, lkt, lmt, grid, k_W, pmf)

        #calculates the dot product between k_W and pmf

        payoff=0.0
        @inbounds for i in hkt:grid
            payoff+= k_W[t+1, i, lkt, lmt] * pmf[hkt, i]
        end
        return payoff
    end

    #lkt+1 = hkt, so we input it here

    function dot_U(t, hkt, grid, U, pkt, pmf)

        #calculates the dot product between U and pmf
        payoff=0.0
        @inbounds for i in hkt:grid
            payoff+= (U[t, i] - pkt) * pmf[hkt, i]
        end
        return payoff
    end

    function compute_payoff_pkt!(t, payoff, pmf, probm, k_W, m_W, share, U, hkt, lmt, Vmax, grid)

        @inbounds for i in 1:(length(share)+1)

            if(i==1)
                pkt = -Vmax
            else
                pkt = U[t, hkt] * share[i-1]
            end    

            cmt = cutoff_manual(t, pkt, hkt, lmt, m_W, grid)

            if(cmt==grid+2) 
                #means that pkt<0, so it will be always rejected
                #lmt+1 == lmt
                payoff[i] += dot_W(t, hkt, hkt, lmt, grid, k_W, pmf)



            elseif(cmt == grid+1) 
                #means that cutoff doesn't exist, so the payment offer will be accepted by all θmt+1

                #since sum(probm[lmt:grid])==1, we didn't include it here
                payoff[i] += dot_U(t, hkt, grid, U, pkt, pmf)

            else
                #cases where cmt is between 1 and grid

                if(cmt < lmt)
                    #means that pkt<minimum continuation value of m, so it will be always rejected
                    #similar to pkt<0
                    #lmt+1 == lmt
                    payoff[i] += dot_W(t, hkt, hkt, lmt, grid, k_W, pmf)

                elseif(cmt==lmt)

                    #accepted only if hmt+1==lmt
                    payoff[i] += dot_U(t, hkt, grid, U, pkt, pmf) * probm[lmt]

                    #rejected otherwise
                    payoff[i] += dot_W(t, hkt, hkt, lmt, grid, k_W, pmf) * sum(@view(probm[(lmt+1):grid]))

                else
                    #cmt > lmt
                    #accepted if hmt+1 <=cmt
                    payoff[i] += dot_U(t, hkt, grid, U, pkt, pmf) * sum(@view(probm[lmt:cmt]))


                    #rejected otherwise
                    #lmt+1 = cmt
                    payoff[i] += dot_W(t, hkt, hkt, cmt, grid, k_W, pmf) * sum(@view(probm[(cmt+1):grid]))

                end
            end
        end
    end

    grid_payment=grid_payment
    share = LinRange(hlow, hhigh-hlow, grid_payment)

    #testing cutoff in this newer version of Pkt

    #function to calculate the optimal payment offer and directly populate the array

    function Pkt!(t, hkt, lmt, k_W, m_W, k_L, Pkt_array, pmf, pmf2, share, U, Vmax=Vmax, grid=grid)


        #pmf of m is a function of t (at t=0 (index 1), the information is symmetric)
        #array: pmf[lmt, 1:end] or pmf2[lmt, 1:end]
        probm = zeros(grid)
        populate_probm!(t, lmt, probm, pmf, pmf2, grid)

        payoff = zeros(length(share)+1)
        compute_payoff_pkt!(t, payoff, pmf, probm, k_W, m_W, share, U, hkt, lmt, Vmax, grid)

        #payoffs matrix####

        payoff_reorg, index_reorg = findmax(payoff)

        if(index_reorg==1)
                payment = -Vmax
            else
                payment = U[t, hkt] * share[index_reorg-1]
            end    


        #calculating the optimal policy between liquidating, reorganizing or waiting ####
        payoff_liq = k_L(t)

        #waiting payoff is associated with the first possible payment, the waiting offer(-Vmax)
        payoff_wait = payoff[1]

        payoff_max, policy = findmax((payoff_liq, payoff_wait, payoff_reorg))



        #populates the array
        Pkt_array[t, hkt, lmt, :] .= payment, cutoff_manual(t, payment, hkt, lmt, m_W, grid), payoff_reorg, payoff_wait, payoff_max, policy

    end





    #optimal proposal
    function propose(t, hkt, lkt, lmt, Pkt_array)


        #it just searches for the corresponding Pkt
        payoff_max, policy = @views Pkt_array[t, hkt, lmt, end-1:end]

        return payoff_max, policy
    end


    #"cdf" for the discretized distribution
    function cdf_discrete(x, lt, pmf)

        #returns the probability that θt takes a value less than or equal to x, given lt
        #if needed, ie in threshold, we can just subtract 0.01 from x, so cdf_discrete the prob that θt < x, strictly
        return sum(@view(pmf[lt, lt:x]))
    end



    #function to calculate the payment offer threshold ϕmt

    function threshold_m(t, lkt, lmt, Pmt_array)

        #searches the index where the optimal policy is either 2.0 or 3.0 (wait or reorganize)
        #"Return the index of the first value in a greater than or equal to x, according to the specified order. 
        #Return length(a) + 1 if x is greater than all values in a. a is assumed to be sorted."
        ts = @views searchsortedfirst(Pmt_array[t, :, lkt, end], 2.0)


        #returns error message if threshold does not exist (101 is returned if missing by searchsortedfirst)
#         @assert ts < 101 "ρ=$ρ, c0=$c0"
        #removed @assert because it is not necessary to calculate the equilibrium of the game

        return ts

    end


    function prob_liq_m(t, ts, lmt, pmf)

        #probability that m will propose liquidation next period

        #at t==0 (index 1), the info is symmetric, so lmt == θmt, thus it becomes the indicator function
        if(t<=1)
            if(lmt >= ts)
                return 0.0
            else
                return 1.0
            end


        else#if t>0(index>1)

            if(lmt >= ts)
                return 0.0
            else
                return cdf_discrete(ts-1,lmt, pmf) #we subtract 1 because we want prob(θt < ts)
            end

        end


    end


    function dot_W2(t, hkt, hkt_next, lkt, lmt, grid, k_W, pmf)

        #calculates the dot product between k_W and pmf
        #modified to include hkt_next, to use in respond_payment

        payoff=0.0
        @inbounds for i in hkt_next:grid
            payoff+= k_W[t+1, i, lkt, lmt] * pmf[hkt, i]
        end
        return payoff
    end

    # function compute_payoff2!(t, payoff, Pkt_grid, cmt, pmf, probm, k_W, m_W, U, hkt, lmt, grid=grid)

    function compute_payoff_pmt(t, Pmt_array, pmf, k_W, hkt, lkt, lmt, grid=grid)


        payoff=zero(0.0)


        @inbounds for lmt_next in lmt:grid
            pmt = Pmt_array[t, lmt_next, lkt, 1]
            ckt = Int64(Pmt_array[t, lmt_next, lkt, 2])

            if(ckt==grid+2)
                #means that the payment offer is negative (waiting offer)
                #the proposal will be rejected for sure, without lowerbound update
    #             payoff += dot(@view(k_W[t+1, hkt:end, lkt, lmt_next]), @view(pmf[hkt, hkt:end])) * pmf[lmt, lmt_next]
                payoff += dot_W2(t, hkt, hkt, lkt, lmt_next, grid, k_W, pmf) * pmf[lmt, lmt_next]


            elseif(ckt==grid+1)
                #means that there is no cutoff, K will accept the payment offer for all θkt+1
    #             payoff += pmt * sum(@view(pmf[hkt, hkt:end])) * pmf[lmt, lmt_next]
                payoff += pmt * pmf[lmt, lmt_next]

            else

                if(ckt < lkt)
                    #the proposal will be rejected for sure, without lowerbound update
    #                 payoff += dot(@view(k_W[t+1, hkt:end, lkt, lmt_next]), @view(pmf[hkt, hkt:end])) * pmf[lmt, lmt_next]

                    payoff += dot_W2(t, hkt, hkt, lkt, lmt_next, grid, k_W, pmf) * pmf[lmt, lmt_next]


                elseif(ckt==lkt)
                    #will accept only if hkt+1==lkt
                    payoff += pmt * pmf[hkt, lkt] * pmf[lmt,lmt_next]

                    #rejects otherwise
    #                 payoff += dot(@view(k_W[t+1, lkt+1:end, lkt, lmt_next]), @view(pmf[hkt, lkt+1:end])) * pmf[lmt, lmt_next]

                    payoff+= dot_W2(t, hkt, lkt+1, lkt, lmt_next, grid, k_W, pmf) * pmf[lmt, lmt_next]


                else
                #now ckt > lkt

                    if(ckt < hkt)
                        #he rejects, but there is an update in the lowerbound lkt+1==ckt
    #                     payoff += dot(@view(k_W[t+1, hkt:end, ckt, lmt_next]), @view(pmf[hkt, hkt:end])) * pmf[lmt, lmt_next]

                        payoff += dot_W2(t, hkt, hkt, ckt, lmt_next, grid, k_W, pmf) * pmf[lmt, lmt_next]

                    else

                        #receives pmt if his skill level is below or equal to the cutoff
                        payoff += pmt * sum(@view(pmf[hkt, hkt:ckt])) * pmf[lmt, lmt_next]


                        #receives continuation value k_W if his skill level greather than the cutoff
    #                     payoff += dot(@view(k_W[t+1, ckt+1:end , ckt, lmt_next]), @view(pmf[hkt, ckt+1:end])) * pmf[lmt, lmt_next]


                        payoff+= dot_W2(t, hkt, ckt+1, ckt, lmt_next, grid, k_W, pmf) * pmf[lmt, lmt_next]

                    end
                end
            end
        end


        return payoff
    end



    #expected payoff of answering a payment proposal

    function respond_payment(t, hkt, lkt, lmt, Pmt_array, k_W, pmf, grid=grid)


        #the minimum level of hmt necessary for m to offer a payment
        #it will be used as hmt
        ts = threshold_m(t, lkt, lmt, Pmt_array)

        #updated lower bound
        lmt = max(ts, lmt)

        return compute_payoff_pmt(t, Pmt_array, pmf, k_W, hkt, lkt, lmt, grid)

    end




    # quase mesma função que foi criada para acelerar Pkt!(), mas aqui não subtrai pkt
    function dot_U2(t, hkt, grid, U, pmf)

        #calculates the dot product between U and pmf
        payoff=0.0
        @inbounds for i in hkt:grid
            payoff+= U[t, i] * pmf[hkt, i]
        end
        return payoff
    end

    #expected payoff of answering a liquidation proposal

    function respond_liq(t, hkt, lkt, lmt, k_L, m_L, pmf, U, grid=grid)

        #payoff_reorg is the payoff of reorganizing while paying the opponent's liquidation value
        #U_{t} (\theta_{t+1}) because the skill level is of the next period
        #dot is matrix multiplication
        payoff_reorg = dot_U2(t, hkt, grid, U, pmf) - m_L(t)

        payoff_liq = k_L(t)

        return max(payoff_liq, payoff_reorg)
    end



    # @code_warntype respond_liq(t, hkt, lkt, lmt, k_L, m_L)

    #expected payoff of being called to respond
    function respond(t, hkt, lkt, lmt, Pmt_array, k_W, k_L, m_L, pmf, U, grid=grid)

        probm_liq = prob_liq_m(t, threshold_m(t, lkt, lmt, Pmt_array), lmt, pmf)


        return (1.0 - probm_liq) * respond_payment(t, hkt, lkt, lmt, Pmt_array, k_W, pmf) + probm_liq * respond_liq(t, hkt, lkt, lmt, k_L, m_L, pmf, U, grid)
    end
    
    
    function populate_pkt_manual!(t, s_W, j_W, s_L, j_L, Pst_array, Pjt_array, pmf, pmf2, share, U, Vmax, grid)

        @inbounds for l in 1:grid
            for h in 1:grid
                Pkt!(t, h, l, s_W, j_W, s_L, Pst_array, pmf, pmf2, share, U, Vmax, grid) #Pst
                Pkt!(t, h, l, j_W, s_W, j_L, Pjt_array, pmf, pmf2, share, U, Vmax, grid) #Pjt
            end
        end

    end

    function populate_wkt_manual!(t, s_W, j_W, s_L, j_L, Pst_array, Pjt_array, pmf, pmf2, share, U, grid)

        for lm in 1:grid
            for lk in 1:grid
               for hk in lk:grid #populates only if hk >= lk, saving time
                    s_W[t, hk, lk, lm] = λj * respond(t, hk, lk, lm, Pjt_array, s_W, s_L, j_L, pmf, U) + (1-λj) * propose(t, hk, lk, lm, Pst_array)[1]
                    j_W[t, hk, lk, lm] = λj * propose(t, hk, lk, lm, Pjt_array)[1] + (1-λj) * respond(t, hk, lk, lm, Pst_array, j_W, j_L, s_L, pmf, U)

                end
            end
        end
    end


    function populate_periods_manual!(tfinal, tbegin, s_W, j_W, s_L, j_L, Pst_array, Pjt_array, pmf, pmf2, share, U, Vmax, grid)

        @time begin


            for t in tfinal:-1:tbegin

                #populating Pkt
                populate_pkt_manual!(t, s_W, j_W, s_L, j_L, Pst_array, Pjt_array, pmf, pmf2, share, U, Vmax, grid)

                #populating Wkt
                populate_wkt_manual!(t, s_W, j_W, s_L, j_L, Pst_array, Pjt_array, pmf, pmf2, share, U, grid)

            end

        end

    end

   ### Populating the whole game
    populate_periods_manual!(T, 1, s_W, j_W, s_L, j_L, Pst_array, Pjt_array, pmf, pmf2, share, U, Vmax, grid)

    
    ### returns the arrays necessary to simulate the game
    return s_W, j_W, Pst_array, Pjt_array, Ds, Dj
    
end


SMM

In [6]:
@everywhere function error_vector(data_moments, simulated_moments)
    #two arrays with the data moments and the simulated moments
    
    return (simulated_moments .- data_moments) ./ data_moments
    
end

In [7]:
@everywhere function criterion(S, N, data_moments, W_hat, game_parameters, simulation_parameters, data1, data2, A1, A2, n1)
    
    #calculate the value of the objective function
    
    
    function simulate_moments(S, N, game_parameters, simulation_parameters, data1, data2, A1, A2, n1)
        


        Results = loop_simulations(S, N, game_parameters, simulation_parameters, data1, data2, A1, A2, n1)

        Moments = zeros(S, 9)

        μ = simulation_parameters[3]
        for s in 1:S
            
            DF = DataFrame()
            DF.payoff_s = Results[s,:,1];
            DF.payoff_j = Results[s,:,2];
            DF.out = Results[s,:,3];
            DF.t = Results[s,:,4];
            DF.observed_proposals = Results[s,:,5];
            DF.Ds = Results[s,:,6];
            DF.Dj = Results[s,:,7];


            DF.outcome = ifelse.(DF.out .==3.0, "R", "L");


            #desfazendo o deslocamento no índice de t
            DF.t = DF.t .- 1.0;

            # DF.incourt = ifelse.(DF.t .> 0.0, "incourt", "precourt");

            INCOURT = filter(DF -> DF.t .> 0.0, DF);
            PRECOURT = filter(DF -> DF.t .<= 0.0, DF);

            PRECOURT_REORG = filter(PRECOURT -> PRECOURT.outcome .== "R", PRECOURT)
            INCOURT_REORG = filter(INCOURT -> INCOURT.outcome .== "R", INCOURT)

            #cálculo dos momentos
            #1. avg log number of months between observed proposals incourt

            mm1 = INCOURT
            mm1 = log.(mm1.t .* μ ./ mm1.observed_proposals)
            mm1 = mean(mm1)

            #2. fraction reorganized given that the case went into court

            mm2 = INCOURT
            mm2 = size(filter(mm2 -> mm2.outcome .== "R", mm2),1)/ size(mm2,1)

            #3. ln duration of court cases in months

            mm3 = INCOURT
            mm3 = filter(mm3 -> mm3.t .> 0.0, mm3) #removendo os casos 0 para não poluir a média
            mm3.t = mm3.t .* μ
            mm3 = mean(log.(mm3.t)) #log here uses exp as base, so it's the same as ln

            #4. fraction of cases incourt
            mm4 = size(INCOURT,1) / size(DF, 1)

            #5. avg recovery rate for senior given precourt REORGANIZATION

            #aqui nós temos Ds, então é mais fácil. Preciso automatizar isso no código quando for tudo escalado

            mm5 = PRECOURT_REORG
            mm5.R_s = mm5.payoff_s ./ mm5.Ds
            mm5 = mean(mm5.R_s)



            #6. avg recovery rate for junior given precourt REORGANIZATION

            #aqui nós temos Ds, então é mais fácil. Preciso automatizar isso no código quando for tudo escalado

            mm6 = PRECOURT_REORG
            mm6.R_j = mm6.payoff_j ./ mm6.Dj
            mm6 = mean(mm6.R_j)


            #7. junior avg fraction gain given incourt REORGANIZATION

            mm7 = INCOURT_REORG
            mm7 = mean(mm7.payoff_j ./ (mm7.payoff_j .+ mm7.payoff_s )) 

            #8. total recovery rate given incourt REORGANIZATION

            mm8 = INCOURT_REORG
            mm8 = mean( mm8.payoff_s .+ mm8.payoff_j)


            #9. slope of the regression of log(recovery rate | incourt reorg) ~ log(duration)
#                 INCOURT_REORG.log_recovery_rate 
            
            x = log.(INCOURT_REORG.payoff_s .+ INCOURT_REORG.payoff_j)
            
            
            
            X = [ones(size(x,1)) x]


#                 INCOURT_REORG.log_duration 
            y = log.(INCOURT_REORG.t .* μ)

            if(size(INCOURT_REORG, 1) == 0)
                mm9 = zero(0.0)
            else

                mm9 = (X' * X) \ (X' * y)
                
                mm9 = mm9[2]
            end
            

            Moments[s,:] .= [mm1, mm2, mm3, mm4, mm5, mm6, mm7, mm8, mm9]
            

            #if there are no INCOURT or no PRECOURT cases, it will return NaN
            #thus, we replace it by zero
            Moments[s,:] .= replace!(Moments[s,:], NaN=> zero(0.0))
            
        end
        
        return vec(mean(Moments, dims=1))
        
    end
    
    
    

        

    function loop_simulations(S, N, game_parameters, simulation_parameters, data1, data2, A1, A2, n1)



        #S is the number of simulations
        #N is the number of observations
        #data1 is the center of the first cluster
        #A1 is the collection of arrays from the first cluster (s_W, j_W, Pst, Pjt)
        #n1 is the number of observations in the first cluster




        #no futuro os argumentos podem ser arrays de arrays, assim ele faz o loop para cada cluster
        Results = zeros(S, N, 7)

        #últimas entradas de Results são os valores das dívidas


        for s in 1:S
            for n in 1:N
                if(n <= n1)
                    Results[s,n,1:5] .= simulate_game(data1, game_parameters, simulation_parameters, A1[1], A1[2], A1[3], A1[4])
                    Results[s,n, end-1] = A1[5]
                    Results[s,n, end] = A1[6]
                else
                    Results[s,n,1:5] .= simulate_game(data2, game_parameters, simulation_parameters, A2[1], A2[2], A2[3], A2[4])
                    Results[s,n, end-1] = A2[5]
                    Results[s,n, end] = A2[6]
                end

            end
        end

        return Results

    end
    
    
    
    function simulate_game(data, game_parameters, simulation_parameters, s_W=s_W, j_W=j_W, Pst_array=Pst_array, Pjt_array=Pjt_array, t=1, grid=100)


        Vmax = data[1]
        L = data[2]
        Dj = data[3]
        Ds = one(1.0) - Dj


        c0 = game_parameters[3];
        c1 = c0/30;

        ρ = game_parameters[1];
        λj = game_parameters[4];



    #     hst = Hs0
    #     hjt = Hj0

        hst = Int64(simulation_parameters[1])
        hjt = Int64(simulation_parameters[2])

        #assumindo que lower bounds nos períodos iniciais são as próprias habilidades iniciais
        lst = hst
        ljt = hjt

        result = zeros(5);

        #number of observed proposals
        observed_proposals = 0.0


        #recovering "T+1" from s_W 
        T = size(s_W,1)


        while(result[1]==0.0 && t < T)


            u = rand()

            if(u < λj)
                propositor = "j"

            else
                propositor = "s"

            end


            #setting the default variables according to the propositor
            Pkt_array, Cont_val, prop_index, respondent_index, m_L, hkt, lkt, hmt, lmt, lk_next, hk_next, hm_next = choose_parameters(propositor, hst, lst, hjt, ljt, Pst_array, Pjt_array, s_W, j_W)

            #proposal ####
            policy, payoff_prop, payment, lm_next = proposal(Pkt_array, t, hkt, lmt)

            #lm_next is the update of the adversary's lower bound
            if(lm_next>=grid+1 || policy!= 3.0) #para evitar update de lowerbound quando proponente não propõe reorg
                lm_next = lmt
            else
                lm_next = Int64(lm_next)
            end



            if(policy==3.0)

                observed_proposals += 1.0

                payoff_respondent, answer = answer_reorg(payment, Cont_val, t, hm_next, lm_next, lk_next)


                if(answer==1.0)

                    result[prop_index] = payoff_prop
                    result[respondent_index] = payoff_respondent
                    result[3] = 3.0
                    result[4] = t
                    result[5] = observed_proposals

                else

                    t+=1

                    hst, lst, hjt, ljt = update_beliefs(propositor, hk_next, hm_next, lk_next, lm_next)
                end

            elseif(policy==2.0)

                t+=1

                hst, lst, hjt, ljt = update_beliefs(propositor, hk_next, hm_next, lk_next, lm_next)



            else      
                #(policy==1.0)

                observed_proposals += 1.0

                payoff_respondent, answer = answer_liq(m_L, t, hm_next, payoff_prop, L, Ds, Dj, Vmax, ρ, c0, c1)



                if(answer==1.0)

                    result[prop_index] = payoff_prop
                    result[respondent_index] = payoff_respondent
                    result[3] = 1.0
                    result[4] = t
                    result[5] = observed_proposals

                else

                    result[prop_index] = payoff_prop
                    result[respondent_index] = payoff_respondent
                    result[3] = 3.0
                    result[4] = t
                    result[5] = observed_proposals


                end
            end


            if(t==T)

                result[1] = s_W[T, 1, 1, 1]#todos os valores finais de S são iguais, então acessarei o índice 1
                result[2] = j_W[T, 1, 1, 1]
                result[3] = 1.0
                result[4] = t
                result[5] = max(observed_proposals, 1.0)

            end





        end


        return result
    end
    
    function choose_parameters(propositor, hst, lst, hjt, ljt, Pst_array, Pjt_array, s_W, j_W)

        if(propositor=="s")

            Pkt_array = Pst_array
            Cont_val = j_W
            prop_index = 1
            respondent_index = 2 #índice de j, para organizar o payoff
            m_L = j_L

            hkt = hst
            lkt = lst

            hmt = hjt
            lmt = ljt


            lk_next = hkt

            hk_next = draw_beta(hkt)
            hm_next = draw_beta(hmt)

        elseif(propositor=="j")
            Pkt_array = Pjt_array
            Cont_val = s_W
            prop_index = 2
            respondent_index = 1
            m_L = s_L

            hkt = hjt
            lkt = ljt

            hmt = hst
            lmt = lst


            lk_next = hkt


            hk_next = draw_beta(hkt)
            hm_next = draw_beta(hmt)

        else

            println("error: propositor not valid")

        end

        return Pkt_array, Cont_val, prop_index, respondent_index, m_L, hkt, lkt, hmt, lmt, lk_next, hk_next, hm_next
    end


    function draw_beta(hkt, β=game_parameters[2], grid=100)

        u = rand()

        if(hkt == grid)

            return grid

        else

            x = 1.0 - exp(1.0/β * (log(1.0 - u) + β * log(1.0 - hkt/grid)))
            x = round(x * grid, digits=0)
    #         return Int64(x * 100) #to convert in an integer
            return Int64(x)

        end
    end
    
    
    #cost function
    function Ct(t, c0=c0, c1=c1)

        #didn't use D because is normalized to D == 1.0
        #cost at period t=0(index1) is 0
        if(t <= 1)
            return 0
        else
            return c0 + c1 * (t-1) #test to make index==1 be t==0
        end
    end



    #liquidation payoffs
    function s_L(t, L, Ds, Dj, c0, c1)
        return min(L - Ct(t, c0, c1), Ds)
    end


    function j_L(t, L, Ds, Dj, c0, c1)
        return min(L - Ct(t, c0, c1) - s_L(t, L, Ds, Dj, c0, c1), Dj)
    end


    function proposal(Pkt_array, t, hkt, lmt)

        #pkt array será sempre do propositor, quem responder às propostas apenas olhará o seu valor de continuação

        return policy, payoff_prop, payment, lm_next = Pkt_array[t, hkt, lmt, [end,end-1, 1, 2]]

    end


    #maximum value of reorganization each period
    function Vt(Vmax, ρ, t)

        if(t <=1)
            return Vmax
        else
            #(t-2) instead of (t-1) because we shifted the indexes in the game so as to include t==0 at index==1
            return ρ^(t-2) * Vmax
        end


    end

    # answer_liq
    function answer_liq(m_L, t, hm_next, payoff_prop, L, Ds, Dj, Vmax, ρ, c0, c1)

        liq = (m_L(t, L, Ds, Dj, c0, c1), Vt(Vmax, ρ, t) * hm_next/100 - Ct(t, c0, c1) - payoff_prop)

        payoff_liq, answer = findmax(liq)

        return payoff_liq, answer
    end

    function answer_reorg(payment, Cont_val, t, hm_next, lm_next, lk_next)

        reorg_value = (payment, Cont_val[t+1, hm_next, lm_next, lk_next])

        payoff_reorg, answer = findmax(reorg_value)

        return payoff_reorg, answer
    end

    function update_beliefs(propositor, hk_next, hm_next, lk_next, lm_next)

        if(propositor=="s")
            hst = hk_next
            lst = lk_next

            hjt = hm_next
            ljt = lm_next

        else
            hst = hm_next
            lst = lm_next

            hjt = hk_next
            ljt = lk_next
        end


        return hst, lst, hjt, ljt
    end

    
    #simulation####
    simulated_moments = simulate_moments(S, N, game_parameters, simulation_parameters, data1, data2, A1, A2, n1)
    
    
    err = error_vector(data_moments, simulated_moments)
    
    #* in this case represents matrix multiplication
    criterion_val = transpose(err) * W_hat
    criterion_val = criterion_val * err
    
#     return (transpose(err) * W_hat) * err
    return criterion_val
    
end

In [8]:
@everywhere function criterion(S, N, data_moments, W_hat, game_parameters, simulation_parameters, data1, data2, A1, A2, n1)
    
    #calculate the value of the objective function
    
    

LoadError: [91msyntax: incomplete: premature end of input[39m

In [9]:
    function simulate_moments(S, N, game_parameters, simulation_parameters, data1, data2, A1, A2, n1)
        


        Results = loop_simulations(S, N, game_parameters, simulation_parameters, data1, data2, A1, A2, n1)

        Moments = zeros(S, 9)

        μ = simulation_parameters[3]
        for s in 1:S
            
            DF = DataFrame()
            DF.payoff_s = Results[s,:,1];
            DF.payoff_j = Results[s,:,2];
            DF.out = Results[s,:,3];
            DF.t = Results[s,:,4];
            DF.observed_proposals = Results[s,:,5];
            DF.Ds = Results[s,:,6];
            DF.Dj = Results[s,:,7];


            DF.outcome = ifelse.(DF.out .==3.0, "R", "L");


            #desfazendo o deslocamento no índice de t
            DF.t = DF.t .- 1.0;

            # DF.incourt = ifelse.(DF.t .> 0.0, "incourt", "precourt");

            INCOURT = filter(DF -> DF.t .> 0.0, DF);
            PRECOURT = filter(DF -> DF.t .<= 0.0, DF);

            PRECOURT_REORG = filter(PRECOURT -> PRECOURT.outcome .== "R", PRECOURT)
            INCOURT_REORG = filter(INCOURT -> INCOURT.outcome .== "R", INCOURT)

            #cálculo dos momentos
            #1. avg log number of months between observed proposals incourt

            mm1 = INCOURT
            mm1 = log.(mm1.t .* μ ./ mm1.observed_proposals)
            mm1 = mean(mm1)

            #2. fraction reorganized given that the case went into court

            mm2 = INCOURT
            mm2 = size(filter(mm2 -> mm2.outcome .== "R", mm2),1)/ size(mm2,1)

            #3. ln duration of court cases in months

            mm3 = INCOURT
            mm3 = filter(mm3 -> mm3.t .> 0.0, mm3) #removendo os casos 0 para não poluir a média
            mm3.t = mm3.t .* μ
            mm3 = mean(log.(mm3.t)) #log here uses exp as base, so it's the same as ln

            #4. fraction of cases incourt
            mm4 = size(INCOURT,1) / size(DF, 1)

            #5. avg recovery rate for senior given precourt REORGANIZATION

            #aqui nós temos Ds, então é mais fácil. Preciso automatizar isso no código quando for tudo escalado

            mm5 = PRECOURT_REORG
            mm5.R_s = mm5.payoff_s ./ mm5.Ds
            mm5 = mean(mm5.R_s)



            #6. avg recovery rate for junior given precourt REORGANIZATION

            #aqui nós temos Ds, então é mais fácil. Preciso automatizar isso no código quando for tudo escalado

            mm6 = PRECOURT_REORG
            mm6.R_j = mm6.payoff_j ./ mm6.Dj
            mm6 = mean(mm6.R_j)


            #7. junior avg fraction gain given incourt REORGANIZATION

            mm7 = INCOURT_REORG
            mm7 = mean(mm7.payoff_j ./ (mm7.payoff_j .+ mm7.payoff_s )) 

            #8. total recovery rate given incourt REORGANIZATION

            mm8 = INCOURT_REORG
            mm8 = mean( mm8.payoff_s .+ mm8.payoff_j)


            #9. slope of the regression of log(recovery rate | incourt reorg) ~ log(duration)
#                 INCOURT_REORG.log_recovery_rate 
            
            x = log.(INCOURT_REORG.payoff_s .+ INCOURT_REORG.payoff_j)
            
            
            
            X = [ones(size(x,1)) x]


#                 INCOURT_REORG.log_duration 
            y = log.(INCOURT_REORG.t .* μ)

            if(size(INCOURT_REORG, 1) == 0)
                mm9 = zero(0.0)
            else

                mm9 = (X' * X) \ (X' * y)
                
                mm9 = mm9[2]
            end
            

            Moments[s,:] .= [mm1, mm2, mm3, mm4, mm5, mm6, mm7, mm8, mm9]
            

            #if there are no INCOURT or no PRECOURT cases, it will return NaN
            #thus, we replace it by zero
            Moments[s,:] .= replace!(Moments[s,:], NaN=> zero(0.0))
            
        end
        
        return vec(mean(Moments, dims=1))
        
    end
    

simulate_moments (generic function with 1 method)

In [10]:
    function simulate_moments(S, N, game_parameters, simulation_parameters, data1, data2, A1, A2, n1)
        


        Results = loop_simulations(S, N, game_parameters, simulation_parameters, data1, data2, A1, A2, n1)

        Moments = zeros(S, 9)

        μ = simulation_parameters[3]
        for s in 1:S
            
            DF = DataFrame()
            DF.payoff_s = Results[s,:,1];
            DF.payoff_j = Results[s,:,2];
            DF.out = Results[s,:,3];
            DF.t = Results[s,:,4];
            DF.observed_proposals = Results[s,:,5];
            DF.Ds = Results[s,:,6];
            DF.Dj = Results[s,:,7];


            DF.outcome = ifelse.(DF.out .==3.0, "R", "L");


            #desfazendo o deslocamento no índice de t
            DF.t = DF.t .- 1.0;

            # DF.incourt = ifelse.(DF.t .> 0.0, "incourt", "precourt");

            INCOURT = filter(DF -> DF.t .> 0.0, DF);
            PRECOURT = filter(DF -> DF.t .<= 0.0, DF);

            PRECOURT_REORG = filter(PRECOURT -> PRECOURT.outcome .== "R", PRECOURT)
            INCOURT_REORG = filter(INCOURT -> INCOURT.outcome .== "R", INCOURT)

            #cálculo dos momentos
            #1. avg log number of months between observed proposals incourt

            mm1 = INCOURT
            mm1 = log.(mm1.t .* μ ./ mm1.observed_proposals)
            mm1 = mean(mm1)

            #2. fraction reorganized given that the case went into court

            mm2 = INCOURT
            mm2 = size(filter(mm2 -> mm2.outcome .== "R", mm2),1)/ size(mm2,1)

            #3. ln duration of court cases in months

            mm3 = INCOURT
            mm3 = filter(mm3 -> mm3.t .> 0.0, mm3) #removendo os casos 0 para não poluir a média
            mm3.t = mm3.t .* μ
            mm3 = mean(log.(mm3.t)) #log here uses exp as base, so it's the same as ln

            #4. fraction of cases incourt
            mm4 = size(INCOURT,1) / size(DF, 1)

            #5. avg recovery rate for senior given precourt REORGANIZATION

            #aqui nós temos Ds, então é mais fácil. Preciso automatizar isso no código quando for tudo escalado

            mm5 = PRECOURT_REORG
            mm5.R_s = mm5.payoff_s ./ mm5.Ds
            mm5 = mean(mm5.R_s)



            #6. avg recovery rate for junior given precourt REORGANIZATION

            #aqui nós temos Ds, então é mais fácil. Preciso automatizar isso no código quando for tudo escalado

            mm6 = PRECOURT_REORG
            mm6.R_j = mm6.payoff_j ./ mm6.Dj
            mm6 = mean(mm6.R_j)


            #7. junior avg fraction gain given incourt REORGANIZATION

            mm7 = INCOURT_REORG
            mm7 = mean(mm7.payoff_j ./ (mm7.payoff_j .+ mm7.payoff_s )) 

            #8. total recovery rate given incourt REORGANIZATION

            mm8 = INCOURT_REORG
            mm8 = mean( mm8.payoff_s .+ mm8.payoff_j)


            #9. slope of the regression of log(recovery rate | incourt reorg) ~ log(duration)
#                 INCOURT_REORG.log_recovery_rate 
            
            x = log.(INCOURT_REORG.payoff_s .+ INCOURT_REORG.payoff_j)
            
            
            
            X = [ones(size(x,1)) x]


#                 INCOURT_REORG.log_duration 
            y = log.(INCOURT_REORG.t .* μ)

            if(size(INCOURT_REORG, 1) == 0)
                mm9 = zero(0.0)
            else

                mm9 = (X' * X) \ (X' * y)
                
                mm9 = mm9[2]
            end
            

            Moments[s,:] .= [mm1, mm2, mm3, mm4, mm5, mm6, mm7, mm8, mm9]
            

            #if there are no INCOURT or no PRECOURT cases, it will return NaN
            #thus, we replace it by zero
            Moments[s,:] .= replace!(Moments[s,:], NaN=> zero(0.0))
            
        end
        
        return vec(mean(Moments, dims=1))
        
    end
    

simulate_moments (generic function with 1 method)

In [11]:
function simulate_moments(S, N, game_parameters, simulation_parameters, data1, data2, A1, A2, n1)



    Results = loop_simulations(S, N, game_parameters, simulation_parameters, data1, data2, A1, A2, n1)

    Moments = zeros(S, 9)

    μ = simulation_parameters[3]
    for s in 1:S

        DF = DataFrame()
        DF.payoff_s = Results[s,:,1];
        DF.payoff_j = Results[s,:,2];
        DF.out = Results[s,:,3];
        DF.t = Results[s,:,4];
        DF.observed_proposals = Results[s,:,5];
        DF.Ds = Results[s,:,6];
        DF.Dj = Results[s,:,7];


        DF.outcome = ifelse.(DF.out .==3.0, "R", "L");


        #desfazendo o deslocamento no índice de t
        DF.t = DF.t .- 1.0;

        # DF.incourt = ifelse.(DF.t .> 0.0, "incourt", "precourt");

        INCOURT = filter(DF -> DF.t .> 0.0, DF);
        PRECOURT = filter(DF -> DF.t .<= 0.0, DF);

        PRECOURT_REORG = filter(PRECOURT -> PRECOURT.outcome .== "R", PRECOURT)
        INCOURT_REORG = filter(INCOURT -> INCOURT.outcome .== "R", INCOURT)

        #cálculo dos momentos
        #1. avg log number of months between observed proposals incourt

        mm1 = INCOURT
        mm1 = log.(mm1.t .* μ ./ mm1.observed_proposals)
        mm1 = mean(mm1)

        #2. fraction reorganized given that the case went into court

        mm2 = INCOURT
        mm2 = size(filter(mm2 -> mm2.outcome .== "R", mm2),1)/ size(mm2,1)

        #3. ln duration of court cases in months

        mm3 = INCOURT
        mm3 = filter(mm3 -> mm3.t .> 0.0, mm3) #removendo os casos 0 para não poluir a média
        mm3.t = mm3.t .* μ
        mm3 = mean(log.(mm3.t)) #log here uses exp as base, so it's the same as ln

        #4. fraction of cases incourt
        mm4 = size(INCOURT,1) / size(DF, 1)

        #5. avg recovery rate for senior given precourt REORGANIZATION

        #aqui nós temos Ds, então é mais fácil. Preciso automatizar isso no código quando for tudo escalado

        mm5 = PRECOURT_REORG
        mm5.R_s = mm5.payoff_s ./ mm5.Ds
        mm5 = mean(mm5.R_s)



        #6. avg recovery rate for junior given precourt REORGANIZATION

        #aqui nós temos Ds, então é mais fácil. Preciso automatizar isso no código quando for tudo escalado

        mm6 = PRECOURT_REORG
        mm6.R_j = mm6.payoff_j ./ mm6.Dj
        mm6 = mean(mm6.R_j)


        #7. junior avg fraction gain given incourt REORGANIZATION

        mm7 = INCOURT_REORG
        mm7 = mean(mm7.payoff_j ./ (mm7.payoff_j .+ mm7.payoff_s )) 

        #8. total recovery rate given incourt REORGANIZATION

        mm8 = INCOURT_REORG
        mm8 = mean( mm8.payoff_s .+ mm8.payoff_j)


        #9. slope of the regression of log(recovery rate | incourt reorg) ~ log(duration)
#                 INCOURT_REORG.log_recovery_rate 

        x = log.(INCOURT_REORG.payoff_s .+ INCOURT_REORG.payoff_j)



        X = [ones(size(x,1)) x]


#                 INCOURT_REORG.log_duration 
        y = log.(INCOURT_REORG.t .* μ)

        if(size(INCOURT_REORG, 1) == 0)
            mm9 = zero(0.0)
        else

            mm9 = (X' * X) \ (X' * y)

            mm9 = mm9[2]
        end


        Moments[s,:] .= [mm1, mm2, mm3, mm4, mm5, mm6, mm7, mm8, mm9]


        #if there are no INCOURT or no PRECOURT cases, it will return NaN
        #thus, we replace it by zero
        Moments[s,:] .= replace!(Moments[s,:], NaN=> zero(0.0))

    end

    return vec(mean(Moments, dims=1))

end

simulate_moments (generic function with 1 method)

In [12]:
function loop_simulations(S, N, game_parameters, simulation_parameters, data1, data2, A1, A2, n1)



    #S is the number of simulations
    #N is the number of observations
    #data1 is the center of the first cluster
    #A1 is the collection of arrays from the first cluster (s_W, j_W, Pst, Pjt)
    #n1 is the number of observations in the first cluster




    #no futuro os argumentos podem ser arrays de arrays, assim ele faz o loop para cada cluster
    Results = zeros(S, N, 7)

    #últimas entradas de Results são os valores das dívidas


    for s in 1:S
        for n in 1:N
            if(n <= n1)
                Results[s,n,1:5] .= simulate_game(data1, game_parameters, simulation_parameters, A1[1], A1[2], A1[3], A1[4])
                Results[s,n, end-1] = A1[5]
                Results[s,n, end] = A1[6]
            else
                Results[s,n,1:5] .= simulate_game(data2, game_parameters, simulation_parameters, A2[1], A2[2], A2[3], A2[4])
                Results[s,n, end-1] = A2[5]
                Results[s,n, end] = A2[6]
            end

        end
    end

    return Results

end



function simulate_game(data, game_parameters, simulation_parameters, s_W=s_W, j_W=j_W, Pst_array=Pst_array, Pjt_array=Pjt_array, t=1, grid=100)


    Vmax = data[1]
    L = data[2]
    Dj = data[3]
    Ds = one(1.0) - Dj


    c0 = game_parameters[3];
    c1 = c0/30;

    ρ = game_parameters[1];
    λj = game_parameters[4];



#     hst = Hs0
#     hjt = Hj0

    hst = Int64(simulation_parameters[1])
    hjt = Int64(simulation_parameters[2])

    #assumindo que lower bounds nos períodos iniciais são as próprias habilidades iniciais
    lst = hst
    ljt = hjt

    result = zeros(5);

    #number of observed proposals
    observed_proposals = 0.0


    #recovering "T+1" from s_W 
    T = size(s_W,1)


    while(result[1]==0.0 && t < T)


        u = rand()

        if(u < λj)
            propositor = "j"

        else
            propositor = "s"

        end


        #setting the default variables according to the propositor
        Pkt_array, Cont_val, prop_index, respondent_index, m_L, hkt, lkt, hmt, lmt, lk_next, hk_next, hm_next = choose_parameters(propositor, hst, lst, hjt, ljt, Pst_array, Pjt_array, s_W, j_W)

        #proposal ####
        policy, payoff_prop, payment, lm_next = proposal(Pkt_array, t, hkt, lmt)

        #lm_next is the update of the adversary's lower bound
        if(lm_next>=grid+1 || policy!= 3.0) #para evitar update de lowerbound quando proponente não propõe reorg
            lm_next = lmt
        else
            lm_next = Int64(lm_next)
        end



        if(policy==3.0)

            observed_proposals += 1.0

            payoff_respondent, answer = answer_reorg(payment, Cont_val, t, hm_next, lm_next, lk_next)


            if(answer==1.0)

                result[prop_index] = payoff_prop
                result[respondent_index] = payoff_respondent
                result[3] = 3.0
                result[4] = t
                result[5] = observed_proposals

            else

                t+=1

                hst, lst, hjt, ljt = update_beliefs(propositor, hk_next, hm_next, lk_next, lm_next)
            end

        elseif(policy==2.0)

            t+=1

            hst, lst, hjt, ljt = update_beliefs(propositor, hk_next, hm_next, lk_next, lm_next)



        else      
            #(policy==1.0)

            observed_proposals += 1.0

            payoff_respondent, answer = answer_liq(m_L, t, hm_next, payoff_prop, L, Ds, Dj, Vmax, ρ, c0, c1)



            if(answer==1.0)

                result[prop_index] = payoff_prop
                result[respondent_index] = payoff_respondent
                result[3] = 1.0
                result[4] = t
                result[5] = observed_proposals

            else

                result[prop_index] = payoff_prop
                result[respondent_index] = payoff_respondent
                result[3] = 3.0
                result[4] = t
                result[5] = observed_proposals


            end
        end


        if(t==T)

            result[1] = s_W[T, 1, 1, 1]#todos os valores finais de S são iguais, então acessarei o índice 1
            result[2] = j_W[T, 1, 1, 1]
            result[3] = 1.0
            result[4] = t
            result[5] = max(observed_proposals, 1.0)

        end





    end


    return result
end

function choose_parameters(propositor, hst, lst, hjt, ljt, Pst_array, Pjt_array, s_W, j_W)

    if(propositor=="s")

        Pkt_array = Pst_array
        Cont_val = j_W
        prop_index = 1
        respondent_index = 2 #índice de j, para organizar o payoff
        m_L = j_L

        hkt = hst
        lkt = lst

        hmt = hjt
        lmt = ljt


        lk_next = hkt

        hk_next = draw_beta(hkt)
        hm_next = draw_beta(hmt)

    elseif(propositor=="j")
        Pkt_array = Pjt_array
        Cont_val = s_W
        prop_index = 2
        respondent_index = 1
        m_L = s_L

        hkt = hjt
        lkt = ljt

        hmt = hst
        lmt = lst


        lk_next = hkt


        hk_next = draw_beta(hkt)
        hm_next = draw_beta(hmt)

    else

        println("error: propositor not valid")

    end

    return Pkt_array, Cont_val, prop_index, respondent_index, m_L, hkt, lkt, hmt, lmt, lk_next, hk_next, hm_next
end


function draw_beta(hkt, β=game_parameters[2], grid=100)

    u = rand()

    if(hkt == grid)

        return grid

    else

        x = 1.0 - exp(1.0/β * (log(1.0 - u) + β * log(1.0 - hkt/grid)))
        x = round(x * grid, digits=0)
#         return Int64(x * 100) #to convert in an integer
        return Int64(x)

    end
end


#cost function
function Ct(t, c0=c0, c1=c1)

    #didn't use D because is normalized to D == 1.0
    #cost at period t=0(index1) is 0
    if(t <= 1)
        return 0
    else
        return c0 + c1 * (t-1) #test to make index==1 be t==0
    end
end



#liquidation payoffs
function s_L(t, L, Ds, Dj, c0, c1)
    return min(L - Ct(t, c0, c1), Ds)
end


function j_L(t, L, Ds, Dj, c0, c1)
    return min(L - Ct(t, c0, c1) - s_L(t, L, Ds, Dj, c0, c1), Dj)
end


function proposal(Pkt_array, t, hkt, lmt)

    #pkt array será sempre do propositor, quem responder às propostas apenas olhará o seu valor de continuação

    return policy, payoff_prop, payment, lm_next = Pkt_array[t, hkt, lmt, [end,end-1, 1, 2]]

end


#maximum value of reorganization each period
function Vt(Vmax, ρ, t)

    if(t <=1)
        return Vmax
    else
        #(t-2) instead of (t-1) because we shifted the indexes in the game so as to include t==0 at index==1
        return ρ^(t-2) * Vmax
    end


end

# answer_liq
function answer_liq(m_L, t, hm_next, payoff_prop, L, Ds, Dj, Vmax, ρ, c0, c1)

    liq = (m_L(t, L, Ds, Dj, c0, c1), Vt(Vmax, ρ, t) * hm_next/100 - Ct(t, c0, c1) - payoff_prop)

    payoff_liq, answer = findmax(liq)

    return payoff_liq, answer
end

function answer_reorg(payment, Cont_val, t, hm_next, lm_next, lk_next)

    reorg_value = (payment, Cont_val[t+1, hm_next, lm_next, lk_next])

    payoff_reorg, answer = findmax(reorg_value)

    return payoff_reorg, answer
end

function update_beliefs(propositor, hk_next, hm_next, lk_next, lm_next)

    if(propositor=="s")
        hst = hk_next
        lst = lk_next

        hjt = hm_next
        ljt = lm_next

    else
        hst = hm_next
        lst = lm_next

        hjt = hk_next
        ljt = lk_next
    end


    return hst, lst, hjt, ljt
end



update_beliefs (generic function with 1 method)

Simulated Annealing

In [13]:
@everywhere function draw_delta(t, n)
    #function to generate draws from delta according to Ingber(1989)
    
    #t é temperatura
    #n é o número de draws, ie, tamanho do vetor
    
    u = rand(n)
    
    y = sign.(u .- 1/2) .* t .* ((1 .+ 1 ./ t).^(abs.(2 .* u .- 1)) .-1)
end

In [14]:
@everywhere function compute_initial_solutions(pids, aux, T, ug, lg, us, ls, g, s, f, S, N, data_moments, W_hat, data1, data2, n1)
   
    
    num_digits = 3;
    
    #we generate new gtest points using mu_inv function
    dg = draw_delta(T, size(g,1)) .* (ug .- lg)

    dg = round.(dg, digits=num_digits)

    gtest = g .+ dg

    #next step is to keep solution within bounds
    gtest = round.(gtest, digits=num_digits)
    
    gtest = (gtest .< lg) .* lg .+ (lg .<= gtest) .* (gtest .<= ug) .* gtest .+ (ug .< gtest) .* ug
    
    #we evaluate the function and the change between the test point and the current point
    #for this we will have to solve the game, since we changed the game_parameters
    A2 = @spawnat aux solve_tree(data2, gtest)
    
    A1 = solve_tree(data1, gtest)
    
    A2 = fetch(A2)
    
    #we generate new stest points using mu_inv function
    ds = draw_delta(T, size(s,1)) .* (us .- ls)

    #we need to round because these are changes in θs0, θj0, which will be used as integers
    ds[1:2] .= round.(ds[1:2], digits=0)

    #ds[3] is μ, which can have 3 digits
    ds[3] = round(ds[3], digits=num_digits)

    stest = s .+ ds

    #next step is to keep solution within bounds
    #this is just a multiplication using indicator functions

    stest = round.(stest, digits=num_digits)


    stest = (stest .< ls) .* ls .+ (ls .<= stest) .* (stest .<= us) .* stest .+ (us .< stest) .* us


    #now we evaluate the criterion function directly
    #we don't need to solve the model again because we didn't change the game parameters

    #evaluating the criterion function
    #gtest = game_parameters in this test
    #stest = simulation parameters in this test

    ftest = f(S, N, data_moments, W_hat, gtest, stest, data1, data2, A1, A2, n1)
    
    return ftest, gtest, stest
    
end

In [15]:
function fill_initial_solutions!(pids, pool, T, ug, lg, us, ls, FCvals, Gvals, Svals, g, s, f, S, N, data_moments, W_hat, data1, data2, n1)
    
    
    @time begin

        @sync for (i, w) in enumerate(pool)
            @async FCvals[i], Gvals[i, :], Svals[i, :] = @fetchfrom w compute_initial_solutions(pids, pids[i+length(pool)], T, ug, lg, us, ls, g, s, f, S, N, data_moments, W_hat, data1, data2, n1)
        end
        
        
    end
    
    
end


fill_initial_solutions! (generic function with 1 method)

In [16]:
@everywhere function compute_solutions(pids, aux, T, ug, lg, us, ls, fc, g, s, gamma, Tac, max_energy, f, S, N, data_moments, W_hat, data1, data2, n1)
    
    
    num_digits = 3;
    
    #we generate new gtest
    dg = draw_delta(T, size(g,1)) .* (ug .- lg)
    
    gtest = g .+ dg

    #next step is to keep solution within bounds
    gtest = round.(gtest, digits=num_digits)

    gtest = (gtest .< lg) .* lg .+ (lg .<= gtest) .* (gtest .<= ug) .* gtest .+ (ug .< gtest) .* ug
    

    #we evaluate the function and the change between the test point and the current point
    #for this we will have to solve the game, since we changed the game_parameters
    A2 = @spawnat aux solve_tree(data2, gtest)
    
    A1 = solve_tree(data1, gtest)
    
    A2 = fetch(A2)
    
    
#     A1 = @async @fetchfrom aux solve_tree(data1, gtest)
#     A2 = @sync solve_tree(data2, gtest)

    #now we start another loop to make changes in s, the simulation parameters
    for t2 in 1:(length(pids)*5) #the cost to test simulaton parameters is lower


        #we generate new stest points using mu_inv function
        ds = draw_delta(T, size(s,1)) .* (us .- ls)

        #we need to round because these are changes in θs0, θj0, which will be used as integers
        ds[1:2] .= round.(ds[1:2], digits=0)

        #ds[3] is μ, which can have 3 digits
        ds[3] = round(ds[3], digits=num_digits)

        stest = s .+ ds

        #next step is to keep solution within bounds
        #this is just a multiplication using indicator functions
        
        stest = round.(stest, digits=num_digits)
        
        stest = (stest .< ls) .* ls .+ (ls .<= stest) .* (stest .<= us) .* stest .+ (us .< stest) .* us
        

        #now we evaluate the criterion function directly
        #we don't need to solve the model again because we didn't change the game parameters

        #evaluating the criterion function
        #gtest = game_parameters in this test
        #stest = simulation parameters in this test

        ftest = f(S, N, data_moments, W_hat, gtest, stest, data1, data2, A1, A2, n1)
        
        
        #accept if ftest < fc or if (ftest > fc and acceptance function says to accept)
        if ((ftest < fc || rand() < acceptance_function(gamma, fc, Tac, max_energy))==true)
            g = gtest
            s = stest
            fc = ftest 
       
        end
    end
    
    return fc, g, s
    
end

In [17]:
function fill_solutions!(pids, pool, T, ug, lg, us, ls, FCvals, Gvals, Svals, gamma, Tac, max_energy, f, S, N, data_moments, W_hat, data1, data2, n1)
    #     FCvals = SharedArray{Float64}(length(pids))
    #     Gvals = SharedArray{Float64, 2}(length(pids), size(g,1))
    #     Svals = SharedArray{Float64, 2}(length(pids), size(s,1))
        #these values come from the master worker, no need to re-create arrays inside here
    
    
    #cada trabalhador vai trabalhar usando a sua solução anterior.
    @time begin

        @sync for (i, w) in enumerate(pool)
            @async FCvals[i], Gvals[i, :], Svals[i, :] = @fetchfrom w compute_solutions(pids, pids[i+length(pool)], T, ug, lg, us, ls, FCvals[i], Gvals[i, :], Svals[i, :], gamma, Tac, max_energy, f, S, N, data_moments, W_hat, data1, data2, n1)
        end
    
        
    end
    
end


fill_solutions! (generic function with 1 method)

In [18]:
function compute_gamma(FCvals, max_energy, Tac)
    
    #FCvals = array with the values of the criterion function. Example: E(xi) for every i in 1, ..., m
    #max_energy = maximum energy from last round, calculated from FCvals
    #Tac = acceptance temperatura
    
    gamma = zero(0.0)
    
    for i in FCvals
        gamma += exp((i - max_energy)/Tac)
    end
    
    return gamma
end

    
    

compute_gamma (generic function with 1 method)

In [19]:
#analisando acceptance_function primeiro

@everywhere function acceptance_function(gamma, fc, Tac, max_energy)
    
    #gamma is the gamma calculated as eq(15) in Xavier et al. (2010) paper.
    #max_energy is the maximum value of the criterion function among all the workers]
    #fc is the current energy level, i.e., E(xi)
    #Tac is the acceptance temperature
    
    
    A = (fc - max_energy)/Tac
    A = exp(A)
    A = A/gamma
    
    return A
    
end




In [20]:
function compute_var(m, gamma, FCvals, Tac, max_energy)
    
    #m is the number of workers, length(workers())
    
    #compute the variation in the acceptance probabilities
    
    A = zero(0.0)
    
    for fc in FCvals
        A += acceptance_function(gamma, fc, Tac, max_energy)^2
    end
    
    var = (1/m) * (A) - (1/m^2)
    return var
    
end

compute_var (generic function with 1 method)

In [21]:
function temp_max(tfinal, k, D)
    #k is the number of iterations
    #tfinal is the final temperature
    #D is the dimension of the vector of parameters
    
    #this function returns the initial temperature according to the cooling scheme proposed by Ingber(1989)
    #we use ci = c =  1
    #and Ti = T
    #that is, c and T are constant for all the parameters
    
    t0 = tfinal / ((exp(-k^(1/D))))
    
    return t0
    
    
end 

temp_max (generic function with 1 method)

In [22]:
function coupled_sim_anl(f, S, N, data1, data2, n1, data_moments, W_hat, g0, s0, lg, ug, ls, us, Kmax, T0, alpha, pids)
    
    #SA algorithm adapted from 
    #https://www.mathworks.com/matlabcentral/fileexchange/33109-simulated-annealing-optimization
    
    #inputs####
    #f : function to evaluate
    #S: number of simulations per observation
    #N: number of observations in the data set
    #data = [Vh/D, L/D, Dj/D]
    #W_hat: weighting matrix for criterion function
    #g0: initial game parameters
    #s0: initial simulation parameters
    #lg and ug: lower and upper bounds for game parameters
    #ls and us: lower and upper bounds for simulation parameters
    #Kmax: maximum number of temperatures
    #ntest: number of test points simulated for each temperature
    #TolFun: parameter of tolerance to help choose the appropriate solutions
    
    
    #T0 initial temperature
    
    
    #outputs####
    #the optimal values of the function and its parameters: f0, g0, s0
    #the best value of the function at each temperature and its parameters: fvals, gvals, svals
    
    @time begin



        #initializing the algorithm ####

        #calculating the initial energy levels
        g = g0
        s = s0
        D = size(g,1) + size(s,1) #size of the parameter vector
        
        
        lengthworkers = Int64(length(pids)/2)
        
        
        
        
        
        
        #arrays to store the energy level (function val), the g and s for each worker
#         FCvals = SharedArray{Float64}(lengthworkers)
#         Gvals = SharedArray{Float64, 2}(lengthworkers, size(g,1))
#         Svals = SharedArray{Float64, 2}(lengthworkers, size(s,1))
        
        
        FCvals = zeros(lengthworkers)
        Gvals = zeros(lengthworkers, size(g,1))
        Svals = zeros(lengthworkers, size(s,1))
        
        
        #pool of workers, it is half of the pids. 
        #Think of it as the pool of main workers, and the others the auxiliary workers
        pool = pids[1:lengthworkers]
        
        #initial definitions
        T = T0
        Tac = T0 #using the same initial temperature, after a few iterations it will be adjusted automatically
        
        fill_initial_solutions!(pids, pool, T, ug, lg, us, ls, FCvals, Gvals, Svals, g, s, f, S, N, data_moments, W_hat, data1, data2, n1)
    
        

        max_energy = findmax(FCvals[:])[1]
        gamma = compute_gamma(FCvals, max_energy, Tac)


        m = lengthworkers
        var_desired = 0.99 * ((m-1)/m^2)

        #para guardar os melhores valores de cada ciclo de temperatura
        fvals = zeros(Kmax)
        gvals = zeros(Kmax, size(g, 1))
        svals = zeros(Kmax, size(s, 1))


        #Main loop simulates de annealing Kmax times
        for k in 1:Kmax


            #calculating the new guesses
            fill_solutions!(pids, pool, T, ug, lg, us, ls, FCvals, Gvals, Svals, gamma, Tac, max_energy, f, S, N, data_moments, W_hat, data1, data2, n1)

            
            #show values for debugging
#             @show FCvals
#             @show Gvals
#             @show Svals
            
            
            #updates
            max_energy = findmax(FCvals[:])[1]
            gamma = compute_gamma(FCvals, max_energy, Tac)

            var = compute_var(m, gamma, FCvals, Tac, max_energy)

            if(var < var_desired)
                Tac = Tac * (1-alpha)
            else
                Tac = Tac * (1 + alpha)
            end


            #cools the system
            #cooling scheme based on Ingber 1989, as well as the draw_delta() function
            T = T0 * exp(-k^(1/D))


            #selecionando os melhores valores do teste
            fc, index = findmin(FCvals[:])
            g = Gvals[index, :]
            s = Svals[index, :]

            #record the value of the best solution of each cycle
            fvals[k] = fc
            gvals[k, :] .= g
            svals[k, :] .= s

            #message so that we can see the upgrade
            println(" end of cycle = $k, g = $g, s = $s, fc = $fc")



            end
        
         #end of @time
    end
    

    return fvals, gvals, svals
    
end
        
       


coupled_sim_anl (generic function with 1 method)

In [23]:
f = criterion
S = 100
N = 66

#Vh/D, L/D, Dj/D
# data = [1.32, 0.32, 0.78]; 

#data_moments
#the last one is calculated in "LinearRegression"
data_moments = [0.728, 0.843, 2.76, 0.969, 0.6235, 0.5569, 0.773, 0.497, -0.113];

#trocando todos os momentos pelos de dou et al
# data_moments = [1.769, 0.910, 2.571, 0.730, 0.878, 0.221, 0.270, 0.370]


W_hat = Matrix{Int}(I, 9, 9);

#pesos obtidos por meio de bootstrap
# weights = [67.227, 485.003, 382.895, 2251.53, 60.4622, 32.3676, 1035.0, 1469.69]

# W_hat = W_hat .* weights

In [24]:
#ρ, β, c0, λj###
#bounds for g
lg = [0.4, 1.0, 0.01, 0.01]
ug = [0.93, 16.0, 0.18, 0.99]


g0 = lg .+ (ug .- lg)./2
#ρ was limited at 0.93 because a higher value would create more periods and calculations would take longer than we could afford.


#θs0, θj0, μ###
#bounds for s
#μ can be 0.5 month or 18 months
ls = [10, 10, 0.5] 
us = [50, 50, 36]


s0 = ls .+ (us .- ls)./2

Kmax = 2

2

In [25]:
alpha = 0.05
T0 = temp_max(0.001, Kmax, size(g0,1)+size(s0,1))

0.003016476757278734

In [26]:
#creating workers and assigning a seed to each one
pids = workers();

@everywhere using Random

startseed = 2020

for (i, w) in enumerate(pids)
    @spawnat w Random.seed!(startseed+i)
end


In [27]:
#Vh/D, L/D, Dj/D
data1 = [0.86, 0.19, 0.8];
n1 = 49

data2 = [2.65, 0.69, 0.74];
n2 = 17

17

In [28]:
N = 10000
x = rand(N)
X = [ones(N) x]



10000×2 Array{Float64,2}:
 1.0  0.27568
 1.0  0.815658
 1.0  0.336548
 1.0  0.424216
 1.0  0.432429
 1.0  0.108743
 1.0  0.330564
 1.0  0.574496
 1.0  0.714698
 1.0  0.460115
 1.0  0.0155186
 1.0  0.852322
 1.0  0.0713598
 ⋮    
 1.0  0.0989148
 1.0  0.621638
 1.0  0.212856
 1.0  0.95304
 1.0  0.264328
 1.0  0.835286
 1.0  0.0337333
 1.0  0.233146
 1.0  0.482602
 1.0  0.983902
 1.0  0.938762
 1.0  0.864218

In [29]:
y = 10 .+ x .* 0.3

function linreg1(y, X)
    β_hat = (X' * X) \ X' * y
    return(β_hat)
end

function linreg2(y, X)
    β_hat = X \ y
    return(β_hat)
end

linreg1(y,X)

linreg2(y,X)[2]

0.29999999999999993

Dissecando a função de simular momentos



In [30]:
aux = pids[2]

A2 = @spawnat aux solve_tree(data2, g0)
    
A1 = solve_tree(data1, g0)

A2 = fetch(A2)

       From worker 3:	 40.644378 seconds (142.85 M allocations: 3.216 GiB, 1.14% gc time)
42.039702 seconds (142.82 M allocations: 3.214 GiB, 0.98% gc time)


([0.2807000423854271 0.28285510325155144 … 2.2210181313248682 2.2496072163803413; 0.26263938273728976 0.2640211150897802 … 1.9656948051815744 1.9908698659881863; … ; 0.26000000000496165 0.26000000000496176 … 0.34309272811795377 0.3506377484589816; 0.26 0.26 … 0.26 0.26]

[0.0 0.28285510325155144 … 2.2210181313248682 2.2496072163803413; 0.0 0.26406573950001044 … 1.9656948051815744 1.9908698659881863; … ; 0.0 0.26000000000496176 … 0.34309272811795377 0.3506377484589816; 0.26 0.26 … 0.26 0.26]

[0.0 0.0 … 2.2210181313248682 2.2496072163803413; 0.0 0.0 … 1.9656948051815744 1.9908698659881863; … ; 0.0 0.0 … 0.34309272811795377 0.3506377484589816; 0.26 0.26 … 0.26 0.26]

...

[0.0 0.0 … 2.2210181313248682 2.2496072163803413; 0.0 0.0 … 2.1477239753346207 2.1746822627910953; … ; 0.0 0.0 … 0.34309272811795377 0.3506377484589816; 0.26 0.26 … 0.26 0.26]

[0.0 0.0 … 2.2210181313248682 2.2496072163803413; 0.0 0.0 … 2.1556361269718094 2.1826751347860003; … ; 0.0 0.0 … 0.34309272811795377 0.350637748

In [31]:
game_parameters = g0
simulation_parameters = s0

3-element Array{Float64,1}:
 30.0
 30.0
 18.25

In [32]:
Results = loop_simulations(S, N, game_parameters, simulation_parameters, data1, data2, A1, A2, n1)

Moments = zeros(S, 9)

μ = simulation_parameters[3]

s = 1


DF = DataFrame()
DF.payoff_s = Results[s,:,1];
DF.payoff_j = Results[s,:,2];
DF.out = Results[s,:,3];
DF.t = Results[s,:,4];
DF.observed_proposals = Results[s,:,5];
DF.Ds = Results[s,:,6];
DF.Dj = Results[s,:,7];


DF.outcome = ifelse.(DF.out .==3.0, "R", "L");


#desfazendo o deslocamento no índice de t
DF.t = DF.t .- 1.0;

# DF.incourt = ifelse.(DF.t .> 0.0, "incourt", "precourt");

INCOURT = filter(DF -> DF.t .> 0.0, DF);
PRECOURT = filter(DF -> DF.t .<= 0.0, DF);

PRECOURT_REORG = filter(PRECOURT -> PRECOURT.outcome .== "R", PRECOURT)
INCOURT_REORG = filter(INCOURT -> INCOURT.outcome .== "R", INCOURT)

#cálculo dos momentos
#1. avg log number of months between observed proposals incourt

mm1 = INCOURT
mm1 = log.(mm1.t .* μ ./ mm1.observed_proposals)
mm1 = mean(mm1)

#2. fraction reorganized given that the case went into court

mm2 = INCOURT
mm2 = size(filter(mm2 -> mm2.outcome .== "R", mm2),1)/ size(mm2,1)

#3. ln duration of court cases in months

mm3 = INCOURT
mm3 = filter(mm3 -> mm3.t .> 0.0, mm3) #removendo os casos 0 para não poluir a média
mm3.t = mm3.t .* μ
mm3 = mean(log.(mm3.t)) #log here uses exp as base, so it's the same as ln

#4. fraction of cases incourt
mm4 = size(INCOURT,1) / size(DF, 1)

#5. avg recovery rate for senior given precourt REORGANIZATION

#aqui nós temos Ds, então é mais fácil. Preciso automatizar isso no código quando for tudo escalado

mm5 = PRECOURT_REORG
mm5.R_s = mm5.payoff_s ./ mm5.Ds
mm5 = mean(mm5.R_s)



#6. avg recovery rate for junior given precourt REORGANIZATION

#aqui nós temos Ds, então é mais fácil. Preciso automatizar isso no código quando for tudo escalado

mm6 = PRECOURT_REORG
mm6.R_j = mm6.payoff_j ./ mm6.Dj
mm6 = mean(mm6.R_j)


#7. junior avg fraction gain given incourt REORGANIZATION

mm7 = INCOURT_REORG
mm7 = mean(mm7.payoff_j ./ (mm7.payoff_j .+ mm7.payoff_s )) 

#8. total recovery rate given incourt REORGANIZATION

mm8 = INCOURT_REORG
mm8 = mean( mm8.payoff_s .+ mm8.payoff_j)

#9. slope of the regression of log(recovery rate | incourt reorg) ~ log(duration)
#                 INCOURT_REORG.log_recovery_rate 

x = INCOURT_REORG.payoff_s

x[x .< 0.0]

Float64[]

In [33]:
INCOURT

,payoff_s,payoff_j,out,t,observed_proposals,Ds,Dj,outcome
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,String
1,0.0823411,0.070488,3.0,3.0,4.0,0.2,0.8,R
2,0.205912,0.0684652,3.0,1.0,2.0,0.2,0.8,R
3,0.243404,0.0707035,3.0,1.0,2.0,0.2,0.8,R
4,0.360414,0.0829091,3.0,1.0,2.0,0.2,0.8,R
5,0.136974,0.00123688,3.0,3.0,4.0,0.2,0.8,R
6,0.198571,0.0691232,3.0,1.0,2.0,0.2,0.8,R
7,0.106766,0.167185,3.0,2.0,3.0,0.2,0.8,R
8,0.143842,0.0714105,3.0,1.0,2.0,0.2,0.8,R
9,0.157673,0.0936829,3.0,1.0,2.0,0.2,0.8,R


In [34]:
INCOURT[3,:]

,payoff_s,payoff_j,out,t,observed_proposals,Ds,Dj,outcome
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,String
3,0.243404,0.0707035,3.0,1.0,2.0,0.2,0.8,R



Testando função do valor de liquidação

In [35]:
@show L = data1[2]
@show Dj = data1[3]
@show Ds = 1.0 - Dj

L = data1[2] = 0.19
Dj = data1[3] = 0.8
Ds = 1.0 - Dj = 0.19999999999999996


0.19999999999999996

In [36]:
c0 = game_parameters[3]
c1 = c0/30

0.003166666666666666

In [37]:
function Ct(t, c0=c0, c1=c1)

    #didn't use D because is normalized to D == 1.0
    #cost at period t=0(index1) is 0
    if(t <= 1)
        return 0
    else
        return c0 + c1 * (t-1) #test to make index==1 be t==0
    end
end

Ct (generic function with 3 methods)

In [38]:
function s_L(t, L, Ds, Dj, c0, c1)
    return min(L - Ct(t, c0, c1), Ds)
end

s_L (generic function with 1 method)

In [39]:
s_L(3, L, Ds, Dj, c0, c1)

0.08866666666666669

In [40]:
function j_L(t, L, Ds, Dj, c0, c1)
    return min(L - Ct(t, c0, c1) - s_L(t, L, Ds, Dj, c0, c1), Dj)
end

j_L (generic function with 1 method)

In [41]:
j_L(1, L, Ds, Dj, c0, c1)

0.0

In [42]:
ENV["COLUMNS"] = 100 #para aumentar o número de colunas mostradas
ENV["LINES"] = 100 #para aumentar o número de colunas mostradas

100

In [43]:
function Vt(Vmax, ρ, t)

    if(t <=1)
        return Vmax
    else
        #(t-2) instead of (t-1) because we shifted the indexes in the game so as to include t==0 at index==1
        return ρ^(t-2) * Vmax
    end


end

Vmax = data1[1]
ρ = game_parameters[1]

0.665

In [44]:
t = 2

2

In [45]:
j_L(t, L, Ds, Dj, c0, c1)

0.0

In [46]:
s_L(t, L, Ds, Dj, c0, c1)

0.09183333333333335

In [47]:
Vt(Vmax, ρ, t)  - Ct(t,c0, c1)

0.7618333333333334

In [48]:

#procurando nemo

x = A1[1][5, 1,1,1]



0.08230130245732761

In [55]:
i = 1
result = simulate_game(data1, game_parameters, simulation_parameters, A1[1], A1[2], A1[3], A1[4])
while(result[1] > 0.0 && i < 1000000)
    result = simulate_game(data1, game_parameters, simulation_parameters, A1[1], A1[2], A1[3], A1[4])
    #payoff_s, payoff_j, outcome, t, observed_proposals
    
    i+=1
end

result

5-element Array{Float64,1}:
 0.1288467372539206
 0.14470176688535333
 3.0
 3.0
 3.0

In [56]:
i

1000000

### Bug era a possibilidade de s_L(t) < 0

Vou testar se mudar a definição do último período do jogo para L - Ct(t) > 0 também ajuda

In [ ]:
function max_turns_atual(Vmax, L, ρ)
    t = 0

    #using the specification of the theoretical model
    #we are not considering the change in indexation from t to t+1 yet.
    while(ρ^(t-1) * Vmax > L)
        t = t+1
    end
    return t
end

In [ ]:
max_turns_atual(Vmax, L, ρ)

In [ ]:
function max_turns_restrito(Vmax, L, ρ, c0, c1)
    t = 0
    
    #using the specification of the theoretical model
    #we are not considering the change in indexation from t to t+1 yet
    
    #the condition L - c0 - c1*t >= 0 assures that the liquidation payoffs won't be negative
    
    while( (ρ^(t-1) * Vmax > L) && (L - c0 - c1*t >= 0) )
        t = t+1
    end
    return t
    
end

In [ ]:
max_turns_restrito(Vmax, L, ρ, 0.044, 0.044/30)

Calibrando c0 tal que o jogo tenha mais de um período mesmo com valores altos de c0

In [ ]:
ρ = 0.4

c0 = 0.05
c1 = c0/30

Tmax = max_turns_restrito(Vmax, L, ρ, c0, c1)
    
while(Tmax > 1 && i < 1000)
    Tmax = max_turns_restrito(Vmax, L, ρ, c0, c1)
    
    c0 = c0 + 0.001
end
    
Tmax = max_turns_restrito(Vmax, L, ρ, c0, c1)

In [ ]:
ρ

In [ ]:

c0 = 0.18
c1 = c0/30

Tmax = max_turns_restrito(Vmax, L, ρ, c0, c1)

In [ ]:
t

In [ ]:
s_L(2, L, Ds, Dj, c0, c1)

In [ ]:
X = [ones(size(x,1)) x]


#                 INCOURT_REORG.log_duration 
y = log.(INCOURT_REORG.t .* μ)

if(size(INCOURT_REORG, 1) == 0)
    mm9 = zero(0.0)
else

    mm9 = (X' * X) \ (X' * y)

    mm9 = mm9[2]
end


Moments[s,:] .= [mm1, mm2, mm3, mm4, mm5, mm6, mm7, mm8, mm9]


#if there are no INCOURT or no PRECOURT cases, it will return NaN
#thus, we replace it by zero
Moments[s,:] .= replace!(Moments[s,:], NaN=> zero(0.0))
            

In [ ]:
game_parameters = g0
simulation_parameters = s0

simulate_moments(S, N, g0, s0, data1, data2, A1, A2, n1)

In [ ]:
using DelimitedFiles

writedlm( "fvals.csv",  fvals, ',')
writedlm("gvals.csv", gvals, ',')
writedlm("svals.csv", svals, ',')

In [ ]:
fvals, gvals, svals = coupled_sim_anl(f, S, N, data1, data2, n1, data_moments, W_hat, g0, s0, lg, ug, ls, us, Kmax, T0, alpha, pids);
@show fvals[Kmax]
@show gvals[Kmax, :]
@show svals[Kmax, :]